In [1]:
%load_ext autoreload
%autoreload 2

### Importing files

In [2]:
from spectradb import Database
from spectradb.dataloaders import FTIRDataLoader, FluorescenceDataLoader, NMRDataLoader
from pathlib import Path
import pandas as pd
import sqlite3
import os
from IPython.display import display

In [3]:
# Just a helper function to see what's inside the database
def display_database()-> pd.DataFrame:
    with sqlite3.connect("db.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM measurements", conn)
    conn.close()  # unlike other context managers, turns out sqlite requires manual closing
    return display(df)

### Locating the data folder and creating the data object

In [4]:
path = Path().resolve().parents[0]/"Only Dataloaders"
data_path_ftir = path/"Example.SPA"
data_path_fl = path/"Example.csv"
data_path_nmr = path/"Example.txt"

In [5]:
ftir = FTIRDataLoader(data_path_ftir)

Data generated from FTIR spectrometer
File:             Example


In [6]:
ftir.add_metadata(
    sample_name="Sample X", 
    internal_code="X", 
    collected_by="X", 
    comments="Correct measurement"
)

There are two ways to create/access database: 
1) Using context manager
2) Using the standard class

### 1. Using context manager

In [7]:
with Database("db.sqlite") as db: 
    db.add_sample(ftir)

In [8]:
display_database()

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,1,FTIR_1,FTIR,2024-09-16,Sample X,X,X,Correct measurement,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,...",2025-02-06 10:01:38


### 2. Using class

In [9]:
os.remove("db.sqlite")

In [10]:
db = Database("db.sqlite")
db.open_connection()

db.add_sample(ftir)

In [11]:
display_database()

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,1,FTIR_1,FTIR,2024-09-16,Sample X,X,X,Correct measurement,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,...",2025-02-06 10:01:42


In [12]:
db.close_connection()

### Trying to add the same sample twice

In [13]:
with Database("db.sqlite") as db: 
    db.add_sample(ftir)

┌───────────────────────────────────────────────┐
│      ❗**Duplicate Entry Detected**❗        │
│                                               │
│ The data you're trying to add already exists. │
│ Check the following for uniqueness:           │
│ • Instrument ID                               │
│ • Sample Name                                 │
│ • Internal Sample Code                        │
│                                               │
│ Please update the information and try again.  │
└───────────────────────────────────────────────┘



### Adding more data

In [14]:
fl = FluorescenceDataLoader(data_path_fl)

fl.add_metadata(
    identifier="S1", 
    sample_name="X", # replaces the current name
    internal_code="X", 
    collected_by="X", 
    comments="There was a mistake with this measurement"
)


fl.add_metadata(
    identifier="S2", 
    sample_name="X", 
    internal_code="X", 
    collected_by="X", 
    comments="This is the right measurement"
)

Data generated from Agilent Cary Eclipse fluorescence spectrometer
File: Example
Samples:
+------------------------------------------------------------+
| Identifier | Sample Name                                   |
+------------------------------------------------------------+
| S1         | 19                                            |
| S2         | 20                                            |
| S3         | 21                                            |
| S4         | 22                                            |
+------------------------------------------------------------+


In [15]:
with Database("db.sqlite") as db:
    db.add_sample(fl)

In [7]:
display_database()

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,1,FTIR_1,FTIR,2024-09-16,Sample X,X,X,Correct measurement,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,...",2024-11-13 14:33:54
1,2,FL_1,FL,2024-09-18,X,X,X,There was a mistake with this measurement,"[[2.941176414489746, 2.915452003479004, 0.0, -...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2024-11-13 14:33:55
2,3,FL_2,FL,2024-09-18,X,X,X,This is the right measurement,"[[2.9585800170898438, 8.902077674865723, 0.0, ...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2024-11-13 14:33:55
3,4,FL_3,FL,2024-09-18,21,,,,"[[2.949852466583252, 0.0, 0.0, 3.0395138263702...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2024-11-13 14:33:55
4,5,FL_4,FL,2024-09-18,22,,,,"[[18.348623275756836, 0.0, 0.0, 0.0, 3.0769231...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2024-11-13 14:33:55
5,6,NMR_1,NMR,2024-09-16,X,X,X,X,"[-6039.0, -7892.0, -15370.0, -5578.0, 10705.0,...","{""ppm"": [16.428625106811523, 16.42799949645996...",2024-11-13 14:33:55


In [16]:
with Database("db.sqlite") as db:
    fig = db.create_spectrum(sample_ids=["FL_2", "FL_3"])

In [18]:
fig[0]

### Adding more data

In [19]:
nmr = NMRDataLoader(data_path_nmr)
nmr.add_metadata(
    sample_name="X",
    internal_code="X",
    collected_by="X",
    comments="X"
)

Data generated from Bruker NMR (in .txt format)              
File: Example


In [20]:
with Database("db.sqlite") as db:
    db.add_sample(nmr)

In [21]:
display_database()

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,1,FTIR_1,FTIR,2024-09-16,Sample X,X,X,Correct measurement,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,...",2025-02-06 10:01:42
1,2,FL_1,FL,2024-09-18,X,X,X,There was a mistake with this measurement,"[[2.941176414489746, 2.915452003479004, 0.0, -...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13
2,3,FL_2,FL,2024-09-18,X,X,X,This is the right measurement,"[[2.9585800170898438, 8.902077674865723, 0.0, ...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13
3,4,FL_3,FL,2024-09-18,21,,,,"[[2.949852466583252, 0.0, 0.0, 3.0395138263702...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13
4,5,FL_4,FL,2024-09-18,22,,,,"[[18.348623275756836, 0.0, 0.0, 0.0, 3.0769231...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13
5,6,NMR_1,NMR,2024-09-16,X,X,X,X,"[-6039.0, -7892.0, -15370.0, -5578.0, 10705.0,...","{""ppm"": [16.428625106811523, 16.42799949645996...",2025-02-06 10:03:07


### Getting specific instrument and sample data

In [23]:
with Database("db.sqlite") as db:
    df = db.get_data_by_instrument_and_sample("FL", "21")
    display(df)

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,4,FL_3,FL,2024-09-18,21,,,,"[[2.949852466583252, 0.0, 0.0, 3.0395138263702...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13


### Executing a custom query

In [24]:
with Database("db.sqlite") as db:
    result = db.execute_custom_query("SELECT * FROM measurements")
    data, col = result
df = pd.DataFrame(data, columns=col)
df

,measurement_id,sample_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata,date_added
0,1,FTIR_1,FTIR,2024-09-16,Sample X,X,X,Correct measurement,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,...",2025-02-06 10:01:42
1,2,FL_1,FL,2024-09-18,X,X,X,There was a mistake with this measurement,"[[2.941176414489746, 2.915452003479004, 0.0, -...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13
2,3,FL_2,FL,2024-09-18,X,X,X,This is the right measurement,"[[2.9585800170898438, 8.902077674865723, 0.0, ...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13
3,4,FL_3,FL,2024-09-18,21,,,,"[[2.949852466583252, 0.0, 0.0, 3.0395138263702...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13
4,5,FL_4,FL,2024-09-18,22,,,,"[[18.348623275756836, 0.0, 0.0, 0.0, 3.0769231...","{""Excitation"": [200, 205, 210, 215, 220, 225, ...",2025-02-06 10:02:13
5,6,NMR_1,NMR,2024-09-16,X,X,X,X,"[-6039.0, -7892.0, -15370.0, -5578.0, 10705.0,...","{""ppm"": [16.428625106811523, 16.42799949645996...",2025-02-06 10:03:07
